In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
pd.set_option('display.max_columns',None)



from pandas.api.types import CategoricalDtype

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly
from tqdm import tqdm

sns.set(style='whitegrid')
plt.rcParams['figure.figsize']=(15,5)
%config InlineBackend.figure_format = 'png'
%matplotlib inline

Загружаем данные в датафреймы 

In [3]:
df_category = pd.read_pickle("./df_category.pkl")
df_worst_milk_sales_bad_shops = pd.read_pickle("./df_worst_milk_sales_bad_shops.pkl")
df_best_milk_sales_good_shops = pd.read_pickle("./df_best_milk_sales_good_shops.pkl")

In [4]:
#Исправляем табличку 
df_category.columns = list(map(''.join, df_category.columns.values))

In [5]:
lis_good_shops = df_best_milk_sales_good_shops.index.values.tolist()
lis_bad_shops = df_worst_milk_sales_bad_shops.index.values.tolist()

In [6]:
df_checks_with_cat = pd.read_pickle("./df_checks_with_cat.pkl")

In [7]:
#Add overal check sum for each check item 
df_checks_with_cat['Check_sum'] = df_checks_with_cat['BaseSum'].groupby(df_checks_with_cat['CheckUID']).transform('sum')

In [8]:
#Add milk sales sum for each check item 
df_checks_with_cat['Item_milk_sum'] = df_checks_with_cat['BaseSum'].mul(df_checks_with_cat['name_gr'] == 'Молочная Продукция')

In [9]:
df_checks_with_milk_percent = df_checks_with_cat.groupby(['CheckUID','id_tt_cl','Check_sum']).Item_milk_sum.agg('sum').reset_index()

Классфицицируем чеки по объему и проценту молочки

In [10]:
conditions_m = [
    (df_checks_with_milk_percent['Item_milk_sum'] == 0),
    ((df_checks_with_milk_percent['Item_milk_sum'] > 0) & (df_checks_with_milk_percent['Item_milk_sum'] <= 100)),
    ((df_checks_with_milk_percent['Item_milk_sum'] > 100) & (df_checks_with_milk_percent['Item_milk_sum'] <= 150)),
    ((df_checks_with_milk_percent['Item_milk_sum'] > 150) & (df_checks_with_milk_percent['Item_milk_sum'] <= 200)),
    ((df_checks_with_milk_percent['Item_milk_sum'] > 200) & (df_checks_with_milk_percent['Item_milk_sum'] <= 300)),
    ((df_checks_with_milk_percent['Item_milk_sum'] > 300) & (df_checks_with_milk_percent['Item_milk_sum'] <= 500)),
    ((df_checks_with_milk_percent['Item_milk_sum'] > 500) & (df_checks_with_milk_percent['Item_milk_sum'] <= 1000)),
    ((df_checks_with_milk_percent['Item_milk_sum'] > 1000) & (df_checks_with_milk_percent['Item_milk_sum'] <= 2000)),
    (df_checks_with_milk_percent['Item_milk_sum'] > 2000) 
]

In [11]:

values_m = ['M_0','M_0_100','M_100_150','M_150_200','M_200_300','M_300_500','M_500_1000','M_1000_2000','M_2000']

df_checks_with_milk_percent['milk_sum'] = np.select(conditions_m,values_m )

In [12]:
conditions_m = [
    (df_checks_with_milk_percent['Check_sum'] == 0),
    ((df_checks_with_milk_percent['Check_sum'] > 0) & (df_checks_with_milk_percent['Check_sum'] <= 100)),
    ((df_checks_with_milk_percent['Check_sum'] > 100) & (df_checks_with_milk_percent['Check_sum'] <= 150)),
    ((df_checks_with_milk_percent['Check_sum'] > 150) & (df_checks_with_milk_percent['Check_sum'] <= 200)),
    ((df_checks_with_milk_percent['Check_sum'] > 200) & (df_checks_with_milk_percent['Check_sum'] <= 300)),
    ((df_checks_with_milk_percent['Check_sum'] > 300) & (df_checks_with_milk_percent['Check_sum'] <= 500)),
    ((df_checks_with_milk_percent['Check_sum'] > 500) & (df_checks_with_milk_percent['Check_sum'] <= 1000)),
    ((df_checks_with_milk_percent['Check_sum'] > 1000) & (df_checks_with_milk_percent['Check_sum'] <= 2000)),
    (df_checks_with_milk_percent['Check_sum'] > 2000) 
]
values_m = ['0','0_100','100_150','150_200','200_300','300_500','500_1000','1000_2000','2000']

df_checks_with_milk_percent['revenue'] = np.select(conditions_m,values_m )

Делаем датафрейсы с хорошими и плохими магазиными

In [13]:
df_milk_checks_good_shop = df_checks_with_milk_percent[df_checks_with_milk_percent['id_tt_cl'].isin(lis_good_shops)]
df_milk_checks_bad_shop = df_checks_with_milk_percent[df_checks_with_milk_percent['id_tt_cl'].isin(lis_bad_shops)]

In [14]:
df_milk_checks_good_shop_p = df_milk_checks_good_shop.pivot_table(columns=['revenue'], index=['milk_sum'], values = ['Check_sum'],aggfunc= 'sum',fill_value=0 ).reset_index()

#Разгребаем косяки с индеками
df_milk_checks_good_shop_p = df_milk_checks_good_shop.pivot_table(columns=['revenue'], index=['milk_sum'], values = ['Check_sum'],aggfunc= 'sum',fill_value=0 ).reset_index()
df_milk_checks_good_shop_p.set_index(df_milk_checks_good_shop_p['milk_sum'],inplace=True)
df_milk_checks_good_shop_p = df_milk_checks_good_shop_p['Check_sum']
df_milk_checks_good_shop_p = df_milk_checks_good_shop_p.div(df_milk_checks_good_shop_p.values.sum()).mul(100)
del df_milk_checks_good_shop_p ['0']

#Sooty in proper order 
df_milk_checks_good_shop_p = df_milk_checks_good_shop_p[['0_100','100_150','150_200','200_300','300_500','500_1000','1000_2000','2000']]
df_milk_checks_good_shop_p.reindex(['M_0','M_0_100','M_100_150','M_150_200','M_200_300','M_300_500','M_500_1000','M_1000_2000','M_2000'])

revenue,0_100,100_150,150_200,200_300,300_500,500_1000,1000_2000,2000
milk_sum,,,,,,,,
M_0,1.527746,1.880996,2.514972,5.306399,10.355499,15.509736,9.361577,2.913459
M_0_100,0.819070,0.379304,0.550705,1.221020,2.620271,4.739965,3.460937,1.054406
M_100_150,0.000000,0.659078,0.210668,0.555389,1.197147,2.545312,2.247122,0.776702
M_150_200,0.000000,0.000000,0.572569,0.383146,0.890152,1.977681,1.852195,0.833842
M_200_300,0.000000,0.000000,0.000000,0.940252,1.062550,2.596572,2.830664,1.456277
M_300_500,0.000000,0.000000,0.000000,0.000000,1.053135,2.085634,2.790951,1.906170
M_500_1000,0.000000,0.000000,0.000000,0.000000,0.000000,0.868261,1.434390,1.606980
M_1000_2000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.147721,0.271820
M_2000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031559


In [15]:
df_milk_checks_bad_shop_p = df_milk_checks_bad_shop.pivot_table(columns=['revenue'], index=['milk_sum'], values = ['Check_sum'],aggfunc= 'sum',fill_value=0 ).reset_index()

#Разгребаем косяки с индеками
df_milk_checks_bad_shop_p = df_milk_checks_bad_shop.pivot_table(columns=['revenue'], index=['milk_sum'], values = ['Check_sum'],aggfunc= 'sum',fill_value=0 ).reset_index()
df_milk_checks_bad_shop_p.set_index(df_milk_checks_bad_shop_p['milk_sum'],inplace=True)
df_milk_checks_bad_shop_p = df_milk_checks_bad_shop_p['Check_sum']
df_milk_checks_bad_shop_p = df_milk_checks_bad_shop_p.div(df_milk_checks_bad_shop_p.values.sum()).mul(100)
del df_milk_checks_bad_shop_p ['0']

#Sooty in proper order 
df_milk_checks_bad_shop_p = df_milk_checks_bad_shop_p[['0_100','100_150','150_200','200_300','300_500','500_1000','1000_2000','2000']]
df_milk_checks_bad_shop_p.reindex(['M_0','M_0_100','M_100_150','M_150_200','M_200_300','M_300_500','M_500_1000','M_1000_2000','M_2000'])

revenue,0_100,100_150,150_200,200_300,300_500,500_1000,1000_2000,2000
milk_sum,,,,,,,,
M_0,1.587954,2.265473,2.972593,6.123110,11.380624,16.572117,11.862624,5.009808
M_0_100,0.433602,0.288090,0.447528,1.033617,2.122187,4.032508,3.714209,1.758165
M_100_150,0.000000,0.245209,0.112747,0.340064,0.813030,1.955318,2.228955,1.243654
M_150_200,0.000000,0.000000,0.210696,0.186398,0.515978,1.399719,1.861065,1.208692
M_200_300,0.000000,0.000000,0.000000,0.323842,0.533207,1.659285,2.558150,1.960758
M_300_500,0.000000,0.000000,0.000000,0.000000,0.370054,1.071036,2.120297,2.335063
M_500_1000,0.000000,0.000000,0.000000,0.000000,0.000000,0.306605,0.809143,1.670537
M_1000_2000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.056261,0.278574
M_2000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021453


In [16]:
df_milk_checks_dif_shop_p = df_milk_checks_good_shop_p - df_milk_checks_bad_shop_p
df_milk_checks_dif_shop_p = df_milk_checks_dif_shop_p.reindex(['M_0','M_2000','M_1000_2000','M_500_1000','M_300_500','M_200_300','M_150_200','M_100_150','M_0_100'])

In [26]:
df_milk_checks_dif_shop_p.to_pickle("./df_milk_checks_dif_shop_p.pkl")

In [27]:
df_milk_checks_dif_shop_percent = df_milk_checks_dif_shop_p.div(df_milk_checks_bad_shop_p)
df_milk_checks_dif_shop_percent = df_milk_checks_dif_shop_percent.reindex(['M_0','M_2000','M_1000_2000','M_500_1000','M_300_500','M_200_300','M_150_200','M_100_150','M_0_100'])


In [29]:
df_milk_checks_dif_shop_percent.to_pickle("./df_milk_checks_dif_shop_percent.pkl")

In [32]:
df_milk_checks_bad_shop_p = df_milk_checks_bad_shop_p.reindex(['M_0','M_2000','M_1000_2000','M_500_1000','M_300_500','M_200_300','M_150_200','M_100_150','M_0_100'])


In [35]:
df_milk_checks_bad_shop_p.to_pickle("./df_milk_checks_bad_shop_p.pkl")

In [34]:
df_milk_checks_good_shop_p = df_milk_checks_good_shop_p.reindex(['M_0','M_2000','M_1000_2000','M_500_1000','M_300_500','M_200_300','M_150_200','M_100_150','M_0_100'])


In [36]:
df_milk_checks_good_shop_p.to_pickle("./df_milk_checks_good_shop_p.pkl")